In [30]:
import numpy as np #for Data Analysis
import pandas as pd #for scientific computation

In [33]:
from google.colab import drive
drive.mount('/content/grive')

Drive already mounted at /content/grive; to attempt to forcibly remount, call drive.mount("/content/grive", force_remount=True).


In [34]:
df=pd.read_csv('/content/grive/My Drive/amazon_ratings_reviews3.csv')

In [36]:
df.head(27)

,Ratings,Reviews
0,4.1 out of 5,"Not a disappointment, but can be better"
1,3.9 out of 5,Good Product... Though Software Improvements a...
2,4.1 out of 5,"4.5 ⭐Awesome design, good specs and features. ..."
3,4.1 out of 5,Best features and looks in the business
4,4.1 out of 5,"4.5 ⭐Awesome design, good specs and features. ..."
5,4.2 out of 5,***BEST BUDGET SMARTWATCH***
6,4.7 out of 5,It is the best smartwatch iPhone users can get!
7,4.4 out of 5,Fitbit to Samsung Journey. THIS IS AMAZING FOL...
8,4.4 out of 5,DETAILED Review after 1 Month of Use + Battery...
9,3.7 out of 5,"Awesome, affordable Garmin"


In [37]:
df['Ratings'].replace(['4.1 out of 5','3.9 out of 5','4.2 out of 5'],['4 out of 5','4 out of 5','4 out of 5'], inplace = True)

In [38]:
df['Ratings'].replace(['4.7 out of 5','4.4 out of 5','3.7 out of 5'],['5 out of 5','4 out of 5','3 out of 5'], inplace = True)

In [43]:
df['Ratings'].replace(['3.9 out of 5','3.5 out of 5','4.3 out of 5','2.9 out of 5','3.8 out of 5'],['4 out of 5','4 out of 5','4 out of 5','3 out of 5','4 out of 5'], inplace = True)

In [46]:
df.head()

,Ratings,Reviews
0,4 out of 5,"Not a disappointment, but can be better"
1,4 out of 5,Good Product... Though Software Improvements a...
2,4 out of 5,"4.5 ⭐Awesome design, good specs and features. ..."
3,4 out of 5,Best features and looks in the business
4,4 out of 5,"4.5 ⭐Awesome design, good specs and features. ..."


In [47]:
df['Ratings'].value_counts()

4 out of 5    21
NO rating      2
3 out of 5     2
5 out of 5     1
Name: Ratings, dtype: int64

In [49]:
import preprocess_kgptalkie as ps
import re

In [51]:
def get_clean(x):
    x = str(x).lower().replace('\\', '').replace('_', ' ')
    x = ps.cont_exp(x)
    x = ps.remove_emails(x)
    x = ps.remove_urls(x)
    x = ps.remove_html_tags(x)
    x = ps.remove_accented_chars(x)
    x = ps.remove_special_chars(x)
    x = re.sub("(.)\\1{2,}", "\\1", x)
    return x

In [52]:
df['Reviews'] = df['Reviews'].apply(lambda x : get_clean(x))

In [53]:
df.head()

,Ratings,Reviews
0,4 out of 5,not a disappointment but can be better
1,4 out of 5,good product though software improvements are ...
2,4 out of 5,45 awesome design good specs and features miba...
3,4 out of 5,best features and looks in the business
4,4 out of 5,45 awesome design good specs and features miba...


In [54]:
# TFIDF and LinearSVC Algorithm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report

In [55]:
tfidf = TfidfVectorizer(max_features=5, ngram_range=(1,3),analyzer='char')

In [56]:
x=tfidf.fit_transform(df['Reviews'])
y=df['Ratings']

In [57]:
x.shape, y.shape

((26, 5), (26,))

In [58]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2, random_state=0)

In [59]:
x_train.shape

(20, 5)

In [61]:
clf=LinearSVC(C=10,class_weight='balanced')
clf.fit(x_train, y_train)

LinearSVC(C=10, class_weight='balanced', dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [62]:
y_pred=clf.predict(x_test)

In [64]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

  4 out of 5       1.00      0.33      0.50         6
  5 out of 5       0.00      0.00      0.00         0
   NO rating       0.00      0.00      0.00         0

    accuracy                           0.33         6
   macro avg       0.33      0.11      0.17         6
weighted avg       1.00      0.33      0.50         6



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [65]:
x='this product is realy good. i like it'
x=get_clean(x)
vec=tfidf.transform([x])
clf.predict(vec)

array(['4 out of 5'], dtype=object)